In [167]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import json
class WatsonDataAPI:
    def __init__(self, cpd_cluster_host, info_json):
        self.cpd_cluster_host = cpd_cluster_host
        self.token = self.get_token(open(info_json))
    def get_token(self, info_json):
        s = requests.session()
        info = json.load(info_json)
        headers = {
            'cache-control': 'no-cache',
            'content-type': 'application/json'
        }

        payload = json.dumps({"username":info["username"], "password":info["password"]})
        try:
            r = s.post(f'{self.cpd_cluster_host}/icp4d-api/v1/authorize', headers=headers, data=payload, verify=False)
            token=json.loads(r.text)['token']
        except requests.exceptions.RequestException as e:  # This is the correct syntax
            raise SystemExit(e)
        s.close()
        return token

    def get_catalog_id(self, catalog_name):
        s = requests.session()
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token
        }
        try:
            r = s.get(
                f"{self.cpd_cluster_host}/v2/catalogs", 
                verify=False, 
                headers=headers
            )
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)

        catalogs = json.loads(r.text)['catalogs']

        catalog_id = None
        for catalog in catalogs:
            if catalog['entity']['name'] == catalog_name:
                catalog_id = catalog['metadata']['guid']
                break
        if catalog_id is None:
            print(f"The provided catalog name ({catalog_name}) does not exist!")
        s.close()
        return catalog_id
    
    def get_category_id(self, category_path):
        s = requests.session()
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token,
            'Cache-Control': "no-cache",
            'Connection': "keep-alive"
        }
        payload = {
            "_source": ["artifact_id","metadata.name","categories"],
            "query": {
                "bool": {
                    "must":[
                        {"match": {"metadata.name":category_path}},
                        {"match": {"metadata.artifact_type": "category"}}
                    ]

                }
            }
        }
        try:
            r = s.post(
                f"{self.cpd_cluster_host}/v3/search",
                verify=False,
                json=payload,
                headers=headers
            )
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        s.close()
        rows = json.loads(r.text)['rows']

        category_hierarchy = [each.strip() for each in category_path.split('>>')]
        category_name= category_hierarchy[-1]
        is_root = True if len(category_hierarchy)==1 else False
        category_parent_name = category_hierarchy[-2] if not is_root else None

        category_id = None
        for row in rows:
            if row['metadata']['name']== category_name:
                if not is_root and row["categories"]["primary_category_name"]==category_parent_name:
                    category_id = row["artifact_id"]
                    break
                elif is_root:
                    category_id = row["artifact_id"]
                    break
        
        return category_id
    
    def get_asset_id(self, asset_name, catalog_name):
        catalog_id = self.get_catalog_id(catalog_name)
        s = requests.session()
        search_body={
            "query": f"asset.name:{asset_name}"
        }
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token
        }
        try:
            r = s.post(
                f"{self.cpd_cluster_host}/v2/asset_types/asset/search?catalog_id="+catalog_id,
                json=search_body,
                verify=False, 
                headers=headers
            )
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        s.close()
        
        res = json.loads(r.text)['results']
        if len(res)>0:
            asset_id = json.loads(r.text)['results'][0]['metadata']['asset_id']
        else:
            print(f"The provided asset name ({asset_name}) does not exist in catalog name ({catalog_name})!")
            asset_id = None
        return asset_id
    def get_asset_info(self, asset_name, catalog_name):
        catalog_id = self.get_catalog_id(catalog_name)
        asset_id = self.get_asset_id(asset_name, catalog_name)
        
        s = requests.session()
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token
        }
        try:
            r = s.get(
                f"{self.cpd_cluster_host}/v2/assets/{asset_id}?catalog_id={catalog_id}", 
                verify=False, 
                headers=headers
            )
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        r_json = json.loads(r.text)
        print(json.dumps(r_json, indent=4))
        
    
    def create_attribute(self, asset_name, catalog_name):
        catalog_id = self.get_catalog_id(catalog_name)
        asset_id = self.get_asset_id(asset_name, catalog_name)
        
        s = requests.session()
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token
        }
        payload = {
            "name": "column_info",
            "entity":{
            }
        }
        try: 
            r=s.post(
                f"{self.cpd_cluster_host}/v2/assets/{asset_id}/attributes?catalog_id={catalog_id}",
                json=payload,
                headers=headers,
                verify=False
            )
        except requests.exceptions.RequestException as e:
            print('Fail to create attribute')
            raise SystemExit(e)
    def delete_attribute(self, asset_name, catalog_name):
        catalog_id = self.get_catalog_id(catalog_name)
        asset_id = self.get_asset_id(asset_name, catalog_name)
        
        s = requests.session()
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token
        }
        try: 
            r=s.delete(
                f"{self.cpd_cluster_host}/v2/assets/{asset_id}/attributes/column_info?catalog_id={catalog_id}",
                headers=headers,
                verify=False
            )
        except requests.exceptions.RequestException as e:
            print('Fail to delete attribute')
            raise SystemExit(e)
        s.close()
    def get_bizterm_id(self, bizterm, category_name):
        s = requests.session()
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token,
            'Cache-Control': "no-cache",
            'Connection': "keep-alive"
        }

        payload={
            "size": 300, 
            "from": 0, 
            "_source": [
                "artifact_id",
                "metadata.artifact_type",
                "metadata.name",
                "metadata.description",
                "categories",
                "entity.artifacts"],
            "query": {
                "bool": {
                    "filter": {
                        "bool": {
                            "minimum_should_match": 1,
                            "should": [
                                {
                                    "term": {"categories.primary_category_id": category["category_id"]}
                                },
                            ],
                            "must_not": {
                                "terms": {
                                    "metadata.artifact_type": ["category"]
                                }
                            }
                        }
                    }
                }
            }
        }
        try:
            r = s.post(
                f"{self.cpd_cluster_host}/v3/search",
                headers=headers,
                json=search_body,
                verify=False
            )
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        r_json = json.loads(r.text)
        print(json.dumps(r_json, indent=4))
        
        
        
        
        
    def update_attribute(self, asset_name, catalog_name,column_name, bizterm):
        catalog_id = self.get_catalog_id(catalog_name)
        asset_id = self.get_asset_id(asset_name, catalog_name)
        
        s = requests.session()
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer "+self.token
        }
        payload=[
            {
                "op":"add",
                "path":"/"+column_name,
                "value":{
                    "column_terms":[
                        {
                            "term_display_name":bizterm,
                            "term_id":bizterm_id
                        }
                    ]
                }
            }
        ]
        try: 
            r=s.patch(
                f"{self.cpd_cluster_host}/v2/assets/{asset_id}/attributes/column_info?catalog_id={catalog_id}",
                json=payload,
                headers=headers,
                verify=False
            )
        except requests.exceptions.RequestException as e:
            print('Fail to delete attribute')
            raise SystemExit(e)
        
    def map_bizterm_to_asset_in_catalog(self,asset_name, catalog_name, bizterm_name, category_path):
        pass
                    

        
    


In [168]:
api = WatsonDataAPI(cpd_cluster_host = 'https://cpd-zen.apps.infra.cp4dex.com',
                    info_json='info.json')

In [169]:
api.get_asset_id('CDB_P_LN','Test Catalog')

'17ef7b3a-6a1c-492f-b501-8411819c10df'

In [170]:
api.delete_attribute('CDB_P_LN','Test Catalog')

In [171]:
api.get_asset_info('CDB_P_LN','Test Catalog')

{
    "metadata": {
        "usage": {
            "last_updated_at": "2023-02-16T06:19:19Z",
            "last_updater_id": "1000331006",
            "last_update_time": 1676528359139,
            "last_accessed_at": "2023-02-16T06:19:19Z",
            "last_access_time": 1676528359139,
            "last_accessor_id": "1000331006",
            "access_count": 0
        },
        "rov": {
            "mode": 0,
            "collaborator_ids": {},
            "member_roles": {
                "1000331006": {
                    "user_iam_id": "1000331006",
                    "roles": [
                        "OWNER"
                    ]
                }
            }
        },
        "name": "CDB_P_LN",
        "asset_type": "data_asset",
        "origin_country": "none",
        "resource_key": "0000:0000:0000:0000:0000:FFFF:A93E:CE1A|30276|ibmclouddb:/LGHH/CDB_P_LN",
        "rating": 0.0,
        "total_ratings": 0,
        "catalog_id": "32acb64f-fede-475e-93b2-247b53b4b86a",

In [150]:
api.get_asset_info('CDB_P_LN','Test Catalog')

{
    "metadata": {
        "usage": {
            "last_updated_at": "2023-02-14T11:59:57Z",
            "last_updater_id": "1000331006",
            "last_update_time": 1676375997568,
            "last_accessed_at": "2023-02-14T11:59:57Z",
            "last_access_time": 1676375997568,
            "last_accessor_id": "1000331006",
            "access_count": 0
        },
        "rov": {
            "mode": 0,
            "collaborator_ids": {},
            "member_roles": {
                "1000331006": {
                    "user_iam_id": "1000331006",
                    "roles": [
                        "OWNER"
                    ]
                }
            }
        },
        "name": "CDB_P_LN",
        "asset_type": "data_asset",
        "origin_country": "none",
        "resource_key": "0000:0000:0000:0000:0000:FFFF:A93E:CE1A|30276|ibmclouddb:/LGHH/CDB_P_LN",
        "rating": 0.0,
        "total_ratings": 0,
        "catalog_id": "32acb64f-fede-475e-93b2-247b53b4b86a",

In [136]:
# api.get_category_id('CreDB >> SubCategoryA >> Part1')
api.create_column_info_attribute_in_catalog('CDB_P_LN','Test Catalog')


{
    "total_rows": 1,
    "results": [
        {
            "metadata": {
                "usage": {
                    "last_updated_at": "2023-02-14T11:59:57Z",
                    "last_updater_id": "1000331006",
                    "last_update_time": 1676375997568,
                    "last_accessed_at": "2023-02-14T11:59:57Z",
                    "last_access_time": 1676375997568,
                    "last_accessor_id": "1000331006",
                    "access_count": 0
                },
                "rov": {
                    "mode": 0,
                    "collaborator_ids": {},
                    "member_roles": {
                        "1000331006": {
                            "user_iam_id": "1000331006",
                            "roles": [
                                "OWNER"
                            ]
                        }
                    }
                },
                "is_primary_attachment_downloadable": false,
                "name": 

'17ef7b3a-6a1c-492f-b501-8411819c10df'

In [38]:
api.get_catalog_id('Test Catalogs')

The provided catalog name (Test Catalogs) does not exist!


In [28]:
tmp = d.get('a',{}).get('sub_a')

In [30]:
tmp=None

In [33]:
tmp is None

True

In [105]:
tmp= 'adcd'
[each for each in tmp.split('>>')][-1]

'adcd'